In [1]:
!pip3 install tensorflow==2.3.0 tensorflow_hub==0.10.0 matplotlib numpy mlflow==1.10.0 minio boto3==1.9.240

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import boto3
import mlflow
from minio import Minio
from minio.error import ResponseError
import json

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.3.0
Hub version: 0.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU is NOT AVAILABLE


In [3]:
#config
import time
timestr = time.strftime("%Y%m%d%H%M%S")

NGROK_URL='http://mlflowui:5000'
EXPERIMENT='firedetection'
EPOCHS=1
VERSION=1
SAVE_PATH='saved_model'
LOSS=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1)
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'

module_selection = ("mobilenet_v2_100_224", 224) 
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = 32 

SAVE_PATH='saved_model'
%mkdir -p saved_model

Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::{}".format(EXPERIMENT)
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            }

        ]}


In [5]:
def create_s3_bucket():
    minioClient = Minio('minio1:9000',
                  access_key="minio",
                  secret_key="minio123",
                  secure=False)

    print(minioClient.list_buckets())

    try:
        if not minioClient.bucket_exists("mlflowruns"):
            minioClient.make_bucket('mlflowruns')
    except ResponseError as err:
        print(err)

    buckets = minioClient.list_buckets()
    for bucket in buckets:
        print(bucket.name, bucket.creation_date)

    policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            }

        ]}

    minioClient.set_bucket_policy('mlflowruns', json.dumps(policy))

    # List all object paths in bucket that begin with my-prefixname.
    objects = minioClient.list_objects('mlflowruns', prefix='',
                              recursive=True)
    for obj in objects:
        print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
            obj.etag, obj.size, obj.content_type)

In [6]:
create_s3_bucket()

[<minio.definitions.Bucket object at 0x7f5244de71d0>, <minio.definitions.Bucket object at 0x7f5244dd2a10>]
data 2020-12-04 10:06:15.084000+00:00
mlflowruns 2020-12-04 09:42:35.418000+00:00


mlflowruns b'10/54bff8af1b4b4373ab558ab86eb4d3a8/artifacts/mlflowruns/features.txt' 2020-12-04 09:50:30.852000+00:00 b43d25ef83e4d05211495d640dfa8869 54 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/model/MLmodel' 2020-12-04 13:04:04.086000+00:00 0467ffbd468769b7407b077e6f8085bb 314 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/model/conda.yaml' 2020-12-04 13:04:04.185000+00:00 5205ebd78b11e44f4c9308e7ec42c1a3 127 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/model/data/keras_module.txt' 2020-12-04 13:04:05.328000+00:00 65daa80bd048be9a6b0b2f1ba806122a 16 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/model/data/model.h5' 2020-12-04 13:04:05.253000+00:00 03ec2e61a132c07fd082bf3f1b6e2127-2 9288352 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/model_summary.txt' 2020-12-04 13:02:10.671000+00:00 ab14a9972148d128ffcbd9926772e234 759 None
mlflowruns b'12/040e1f057efe456090eac09322b64afe/artifacts/tensor

In [7]:
tags={}
tags['TYPE']='CUSTOM'
tags['TRANSFER_LEARNING']='YES'
tags['TFHUB']='NO'

In [8]:
#Mlflow settings

#set MLflow server 
mlflow.set_tracking_uri(NGROK_URL)

#Set experiment
'''
if mlflow.get_experiment_by_name(EXPERIMENT) != None:
    exp_id = mlflow.set_experiment(EXPERIMENT)
else: 
    exp_id = mlflow.create_experiment(EXPERIMENT)
'''

!mlflow experiments create --experiment-name $EXPERIMENT --artifact-location s3://mlflowruns/

Traceback (most recent call last):
  File "/opt/conda/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mlflow/experiments.py", line 42, in create
    exp_id = store.create_expe

In [9]:
from mlflow import tensorflow
mlflow.tensorflow.autolog()

In [10]:
exp_id = mlflow.set_experiment(EXPERIMENT)

#Close active runs
if mlflow.active_run():
    mlflow.end_run()

In [11]:
data_dir = 'fire_detection/'

In [12]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = False 
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 403 images belonging to 2 classes.


Found 1614 images belonging to 2 classes.


In [13]:
do_fine_tuning = False 

In [14]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4


In [15]:
summary=str(model.summary())
out = open('report.txt','w')
out.write(summary)
out.close

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


<function TextIOWrapper.close()>

In [16]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=LOSS,
  metrics=['accuracy'])

In [17]:
with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False): 
  steps_per_epoch = train_generator.samples // train_generator.batch_size
  validation_steps = valid_generator.samples // valid_generator.batch_size
  
  hist = model.fit(
      train_generator,
      epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
      validation_data=valid_generator,
      validation_steps=validation_steps).history

  #Set tags
  mlflow.set_tags(tags)

  #mlflow logging
  
  # log parameters
  mlflow.log_param("epochs", EPOCHS)
  mlflow.log_param("loss_function", str(LOSS))

  # log metrics
  for epoch in range(0, EPOCHS):
    mlflow.log_metric("accuracy", hist['accuracy'][epoch])
    mlflow.log_metric("loss",  hist['loss'][epoch])
    mlflow.log_metric("val_accuracy",  hist['val_accuracy'][epoch])
    mlflow.log_metric("val_loss",  hist['val_loss'][epoch])
  
  #results=evaluate_model() #TODO
  #mlflow.log_metric("average_loss", results[0])
  #mlflow.log_metric("average_acc", results[1])
  
  #log model
  #model.save(os.path.join(BASE_DIR, "models", "{}.h5".format(int(t)))) #HDF5 format
  tf.saved_model.save(model, SAVE_PATH) #SavedModel format
  
  # log artifacts (matplotlib images for loss/accuracy, etc.)
  #mlflow.log_artifacts('report.txt', "mlflowruns")

  mlflow.end_run()

 1/50 [..............................] - ETA: 0s - loss: 0.6862 - accuracy: 0.6250

Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


 2/50 [>.............................]

 - ETA: 57s - loss: 0.7762 - accuracy: 0.5938

...

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets
